In [4]:
import pandas as pd

In [1]:
import csv
import numpy as np
from faker import Faker

# Initialize the Faker generator
fake = Faker()

# Set the random seed for reproducibility
np.random.seed(42)

# Define the number of samples
num_samples = 20000

# Create empty lists to store the data
data = []

cnt=1

patient_id =10000

# Generate synthetic data
for _ in range(num_samples):
    # Generate patient ID
    # patient_id = fake.unique.random_number(digits=6)

    # Generate patient demographics
    age = np.random.randint(18, 90)
    gender = fake.random_element(['Male', 'Female'])
    location = fake.city()

    # Generate medical history
    medical_condition = fake.random_element(['None', 'Diabetes', 'Hypertension', 'Asthma', 'Cancer'])

    # Generate treatment details
    treatment_type = fake.random_element(['Surgery', 'Medication', 'Physical Therapy'])

    # Generate length of stay
    length = np.random.randint(1, 30)

    # Generate insurance coverage
    insurance = fake.random_element(['Private', 'Public', 'None'])

    # Generate diagnosis
    diagnosis = fake.random_element(['Infection', 'Injury', 'Cardiovascular', 'Respiratory', 'Neurological'])

    # Generate discharge status
    discharge_status = fake.random_element(['Home', 'Another Facility', 'Deceased'])

    # Generate reimbursement rates
    if insurance == 'Private':
        reimbursement = np.random.uniform(0.7, 1.0)
    elif insurance == 'Public':
        reimbursement = np.random.uniform(0.3, 0.6)
    else:
        reimbursement = 0

    # Calculate profit based on the features and their relationships
    if treatment_type == "Medication":
        base_profit = 5000  # Base profit for each patient
    if treatment_type == "Physical Therapy":
        base_profit = 10000  # Base profit for each patient
    if treatment_type == "Surgery":
        base_profit = 20000  # Base profit for each patient
    age_effect = (age - 45) / 10  # Age negatively affects profit after 45 years
    length_of_stay_effect = length / 10  # Longer stays result in higher costs and potentially higher profit
    insurance_effect = 1 if insurance == 'Private' else 0.5  # Private insurance results in higher reimbursement
    profit = base_profit + age_effect + length_of_stay_effect + reimbursement * insurance_effect

    # Append data to the list
    data.append([patient_id, age, gender, location, medical_condition, treatment_type, length, insurance,
                 diagnosis, discharge_status, reimbursement, profit])

    print("Count : ",cnt)
    cnt+=1

    patient_id+=1

# Define the CSV file path
csv_file_path = '..\data\hospital_profit_dataset1.csv'

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header row
    writer.writerow(['Patient ID', 'Age', 'Gender', 'Location', 'Medical Condition', 'Treatment Type',
                     'Length of Stay', 'Insurance Coverage', 'Diagnosis', 'Discharge Status',
                     'Reimbursement Rate', 'Profit'])
    # Write the data rows
    writer.writerows(data)

print(f"Dataset saved successfully as '{csv_file_path}'")


Count :  1
Count :  2
Count :  3
Count :  4
Count :  5
Count :  6
Count :  7
Count :  8
Count :  9
Count :  10
Count :  11
Count :  12
Count :  13
Count :  14
Count :  15
Count :  16
Count :  17
Count :  18
Count :  19
Count :  20
Count :  21
Count :  22
Count :  23
Count :  24
Count :  25
Count :  26
Count :  27
Count :  28
Count :  29
Count :  30
Count :  31
Count :  32
Count :  33
Count :  34
Count :  35
Count :  36
Count :  37
Count :  38
Count :  39
Count :  40
Count :  41
Count :  42
Count :  43
Count :  44
Count :  45
Count :  46
Count :  47
Count :  48
Count :  49
Count :  50
Count :  51
Count :  52
Count :  53
Count :  54
Count :  55
Count :  56
Count :  57
Count :  58
Count :  59
Count :  60
Count :  61
Count :  62
Count :  63
Count :  64
Count :  65
Count :  66
Count :  67
Count :  68
Count :  69
Count :  70
Count :  71
Count :  72
Count :  73
Count :  74
Count :  75
Count :  76
Count :  77
Count :  78
Count :  79
Count :  80
Count :  81
Count :  82
Count :  83
Count :  84
C

In [3]:
import pandas as pd
data = pd.read_csv("..\data\hospital_profit_dataset1.csv")

In [4]:
data.columns

Index(['Patient ID', 'Age', 'Gender', 'Location', 'Medical Condition',
       'Treatment Type', 'Length of Stay', 'Insurance Coverage', 'Diagnosis',
       'Discharge Status', 'Reimbursement Rate', 'Profit'],
      dtype='object')

In [5]:
data.drop('Location',axis=1,inplace=True)

In [ ]:
import seaborn as sns

sns.distplot(data,x=data["Profit"])

In [7]:
cat_columns= ['Gender','Medical Condition','Treatment Type','Insurance Coverage','Diagnosis','Discharge Status']

num_column= [i for i in data.columns if i not in cat_columns]

dummies = pd.get_dummies(data[cat_columns],drop_first=True)

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# import pickle

# pickle.dump(model,open("../scaler/profitability_scalerp.pickle","wb"))

# numeric_columns = pd.DataFrame(scaler.fit_transform(data[num_column]),columns=data[num_column].columns)

new_data = pd.concat([dummies,data[num_column]],axis=1)


In [8]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

target,features = new_data["Profit"],new_data.drop(["Profit"],axis=1)

x_train, x_test, y_train, y_test = train_test_split(features, target)

model = RandomForestRegressor(n_estimators=200)

model.fit(x_train,y_train)

### Evaluating the model

from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error

predicted = model.predict(x_test)

print("R2 Score is : ",r2_score(predicted,y_test))

print("Mean Squared Error is : ",mean_squared_error(predicted,y_test))

print("Mean Absolute Error is : ",mean_absolute_error(predicted,y_test))

import pickle

pickle.dump(model,open("..\models\profitability_model1.pickle","wb"))

R2 Score is :  0.9999999998076673
Mean Squared Error is :  0.007548996831867987
Mean Absolute Error is :  0.06729673227269796


# Prediction

In [11]:
patient_id = 10000

model_pofit = pickle.load(open("..\models\profitability_model1.pickle","rb"))

data_pofit = pd.read_csv("..\data\hospital_profit_dataset1.csv")

data_pofit.drop(['Location','Profit'],axis=1,inplace=True)

def prepare_profitability_sample(patient_id):

    new_data = data_pofit[data_pofit["Patient ID"] == patient_id]

    sample = {'Gender_Male': 0,
    'Medical Condition_Cancer': 0,
    'Medical Condition_Diabetes': 0,
    'Medical Condition_Hypertension': 0,
    'Medical Condition_None': 0,
    'Treatment Type_Physical Therapy': 0,
    'Treatment Type_Surgery': 0,
    'Insurance Coverage_Private': 0,
    'Insurance Coverage_Public': 0,
    'Diagnosis_Infection': 0,
    'Diagnosis_Injury': 0,
    'Diagnosis_Neurological': 0,
    'Diagnosis_Respiratory': 0,
    'Discharge Status_Deceased': 0,
    'Discharge Status_Home': 0,
    'Patient ID': 0,
    'Age': 0,
    'Length of Stay': 0,
    'Reimbursement Rate': 0}

    cat_columns= ['Gender','Medical Condition','Treatment Type','Insurance Coverage','Diagnosis','Discharge Status']

    num_column= [i for i in data_pofit.columns if i not in cat_columns]

    for idx,row in new_data.iterrows():

        for key,value in row.to_dict().items():

            if key in cat_columns:

                column = str(key)+"_"+str(value)

                if column in sample.keys():

                    sample[column] = [1]

            if key in num_column:

                sample[key] = [value]

    return pd.DataFrame(sample)


inp = prepare_profitability_sample(patient_id)

model.predict(inp)[0]
    

10006.037342437965

In [20]:
def get_patient_data_profit(patient_id):

    actual_data = pd.read_csv("..\data\hospital_profit_dataset1.csv")

    actual_data.drop(['Profit'],axis=1,inplace=True)

    patient_data = data_pofit[data_pofit["Patient ID"] == patient_id]

    actual_data = patient_data.to_dict()

    for key, value in actual_data.items():
        actual_data[key] = value[list(value.keys())[0]]

    if not patient_data.empty:
        return actual_data
    else:
        return None
d = get_patient_data_profit(patient_id)